Importing all necessary libraries and pachages

In [25]:
import numpy as np 
import pandas as pd
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tarfile

Creating a dictionary aligning with dataset

In [23]:
dataset_folder_name = 'UTKFace'

TRAIN_TEST_SPLIT = 0.7
IM_WIDTH = IM_HEIGHT = 198

dataset_dict = {
    'race_id': {
        0: 'white', 
        1: 'black', 
        2: 'asian', 
        3: 'indian', 
        4: 'others'
    },
    'gender_id': {
        0: 'male',
        1: 'female'
    }
}

dataset_dict['gender_alias'] = dict((g, i) for i, g in dataset_dict['gender_id'].items())
dataset_dict['race_alias'] = dict((g, i) for i, g in dataset_dict['race_id'].items())

Extracting the databse tar file in present directory

In [24]:
with tarfile.open('UTKFace.tar.gz', "r:gz") as f:
  f.extractall('')
  f.close()
   # Extract all the contents of zip file in current directory

Parsing the files inside the image database, creating a dataframe with specified column name.

In [26]:
def parse_dataset(dataset_path, ext='jpg'):
    """
    Used to extract information about our dataset. It does iterate over all images and return a DataFrame with
    the data (age, gender and sex) of all files.
    """
    def parse_info_from_file(path):
        """
        Parse information from a single file
        """
        try:
            filename = os.path.split(path)[1]
            filename = os.path.splitext(filename)[0]
            age, gender, race, _ = filename.split('_')

            return int(age), dataset_dict['gender_id'][int(gender)], dataset_dict['race_id'][int(race)]
        except Exception as ex:
            return None, None, None
        
    files = glob.glob(os.path.join(dataset_path, "*.%s" % ext))
    
    records = []
    for file in files:
        info = parse_info_from_file(file)
        records.append(info)
        
    df = pd.DataFrame(records)
    df['file'] = files
    df.columns = ['age', 'gender', 'race', 'file']
    df = df.dropna()
    
    return df

Loading the dataframe, reviewing the same with head() function

In [27]:
df = parse_dataset(dataset_folder_name)
df.head()

,age,gender,race,file
0,1.0,male,white,UTKFace/1_0_0_20170110211535352.jpg.chip.jpg
1,60.0,male,black,UTKFace/60_0_1_20170113185153840.jpg.chip.jpg
2,1.0,male,asian,UTKFace/1_0_2_20161219201312292.jpg.chip.jpg
3,29.0,male,others,UTKFace/29_0_4_20170114033029580.jpg.chip.jpg
4,30.0,male,indian,UTKFace/30_0_3_20170119155845146.jpg.chip.jpg


Importing the graphical libraries, and utilizing the graph features, drawing the Pie chart here

In [32]:
import plotly.graph_objects as graph

def plot_distribution(pd_series):
    labels = pd_series.value_counts().index.tolist()
    counts = pd_series.value_counts().values.tolist()
    pie_plot = graph.Pie(labels=labels, values=counts, hole=.3)
    fig = graph.Figure(data=[pie_plot])
    fig.update_layout(title_text='Distribution for %s' % pd_series.name)
    
    fig.show()

Displaying the gender distribution using the pie chart

In [33]:
plot_distribution(df['gender'])